<a href="https://colab.research.google.com/github/brauliosba/TA2-Concurrente/blob/main/TA2_Concurrent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!add-apt-repository ppa:longsleep/golang-backports -y
!apt update
!apt install golang-go
%env GOPATH=/root/go
!go get -u github.com/gopherdata/gophernotes
!cp ~/go/bin/gophernotes /usr/bin/
!mkdir /usr/local/share/jupyter/kernels/gophernotes
!cp ~/go/src/github.com/gopherdata/gophernotes/kernel/* \
       /usr/local/share/jupyter/kernels/gophernotes

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]


In [226]:
%%writefile nn.go
package main

import (
    "encoding/csv"
    "errors"
    "fmt"
    "log"
    "math"
    "math/rand"
    "strconv"
    "time"
    "net/http"
    "gonum.org/v1/gonum/floats"
    "gonum.org/v1/gonum/mat"
)

type neuralNet struct {
    config  neuralNetConfig
    wHidden *mat.Dense
    bHidden *mat.Dense
    wOut    *mat.Dense
    bOut    *mat.Dense
}

type neuralNetConfig struct {
    inputNeurons  int
    outputNeurons int
    hiddenNeurons int
    numEpochs     int
    learningRate  float64
}

var numFolds int = 10
var numClasses int = 3
var numFeatures int = 5

func main() {
    fileDataReal := readArchiveCSV("https://raw.githubusercontent.com/brauliosba/TA2-Concurrente/main//iris.csv")
  	
    config := neuralNetConfig{
        inputNeurons:  4,
        outputNeurons: 3,
        hiddenNeurons: 3,
        numEpochs:     1000,
        learningRate:  0.3,
    }
    network := newNetwork(config)

    fmt.Printf("%d-fold Cross Validation: \n", numFolds)

    for i := 1; i <= numFolds ; i++ {
      trainInputs, trainLabels, testInputs, testLabels := crossValidation(fileDataReal, i)

      if err := network.train(trainInputs, trainLabels); err != nil {
          log.Fatal(err)
      }

      predictions, err := network.predict(testInputs)
      if err != nil {
          log.Fatal(err)
      }

      var truePosNeg int
      numPreds, _ := predictions.Dims()
      for i := 0; i < numPreds; i++ {
          labelRow := mat.Row(nil, i, testLabels)
          var prediction int
          for idx, label := range labelRow {
              if label == 1.0 {
                  prediction = idx
                  break
              }
          }

          if predictions.At(i, prediction) == floats.Max(mat.Row(nil, i, predictions)) {
              truePosNeg++
          }
      }

      accuracy := float64(truePosNeg) / float64(numPreds)

      fmt.Printf("Accuracy = %0.2f\n", accuracy)
    }
}

func crossValidation(dataframeNN [][]float64, currentFold int) (*mat.Dense, *mat.Dense, *mat.Dense, *mat.Dense) {
    var testSize int = len(dataframeNN)/numFolds

    inputsTrainData := make([]float64, (numFeatures - 1)*(len(dataframeNN) - testSize))
    labelsTrainData := make([]float64, numClasses*(len(dataframeNN) - testSize))
    inputsTestData := make([]float64, (numFeatures - 1)*(testSize))
    labelsTestData := make([]float64, numClasses*(testSize))

    var inputsTrainIndex int
    var labelsTrainIndex int
    var inputsTestIndex int
    var labelsTestIndex int

    for idx, record := range dataframeNN {
        if idx < testSize * currentFold && idx >= testSize * (currentFold - 1) {
            for i, val := range record {
                if i == (numFeatures - 1) {
                    classLabel := oneHotEncoding(val)
                    for _, value := range classLabel {
                        labelsTestData[labelsTestIndex] = value
                        labelsTestIndex++
                    }
                    continue
                }

                inputsTestData[inputsTestIndex] = val
                inputsTestIndex++
            }
        } else {
            for i, val := range record {
                if i == (numFeatures - 1) {
                    classLabel := oneHotEncoding(val)
                    for _, value := range classLabel {
                        labelsTrainData[labelsTrainIndex] = value
                        labelsTrainIndex++
                    }
                    continue
                }

                inputsTrainData[inputsTrainIndex] = val
                inputsTrainIndex++
            }
        }
    }
    inputsTrain := mat.NewDense(len(dataframeNN)-testSize, (numFeatures - 1), inputsTrainData)
    labelsTrain := mat.NewDense(len(dataframeNN)-testSize, numClasses, labelsTrainData)
    inputsTest := mat.NewDense(testSize, (numFeatures - 1), inputsTestData)
    labelsTest := mat.NewDense(testSize, numClasses, labelsTestData)
  
    return inputsTrain, labelsTrain, inputsTest, labelsTest
}

func readArchiveCSV(url string) [][]float64 {
    resp, err := http.Get(url)

    if err != nil {
        log.Fatal("No se puede leer la url ", err)
    }
    defer resp.Body.Close()
    csvReader := csv.NewReader(resp.Body)

    fileData, err := csvReader.ReadAll()
    if err != nil {
        log.Fatal("No se puede parsear dataset ", err)
    }

    headers := make([]string, len(fileData[0]))
    copy(headers, fileData[0])

    fileData = fileData[1:]
    fileDataReal := make([][]float64, len(fileData))

    for i := range fileDataReal {
        fileDataReal[i] = make([]float64, len(headers))
        for j := range fileDataReal[i] {
            val, _ := strconv.ParseFloat(fileData[i][j], 64)
            fileDataReal[i][j] = float64(val)
        }
    }

    return fileDataReal
}

func oneHotEncoding(num float64) []float64 {
    classLabel := make([]float64, numClasses)
    if numClasses == 1 {
        classLabel[0] = num
    } else {
        classLabel[int(num)] = 1
    }
    return classLabel
}

func newNetwork(config neuralNetConfig) *neuralNet {
	  return &neuralNet{config: config}
}

func (nn *neuralNet) train(x, y *mat.Dense) error {
    randSource := rand.NewSource(time.Now().UnixNano())
    randGen := rand.New(randSource)

    wHidden := mat.NewDense(nn.config.inputNeurons, nn.config.hiddenNeurons, nil)
    bHidden := mat.NewDense(1, nn.config.hiddenNeurons, nil)
    wOut := mat.NewDense(nn.config.hiddenNeurons, nn.config.outputNeurons, nil)
    bOut := mat.NewDense(1, nn.config.outputNeurons, nil)

    wHiddenRaw := wHidden.RawMatrix().Data
    bHiddenRaw := bHidden.RawMatrix().Data
    wOutRaw := wOut.RawMatrix().Data
    bOutRaw := bOut.RawMatrix().Data

    for _, param := range [][]float64{
        wHiddenRaw,
        bHiddenRaw,
        wOutRaw,
        bOutRaw,
    } {
      for i := range param {
          param[i] = randGen.Float64()
      }
    }

    output := new(mat.Dense)

    if err := nn.backpropagate(x, y, wHidden, bHidden, wOut, bOut, output); err != nil {
        return err
    }

    nn.wHidden = wHidden
    nn.bHidden = bHidden
    nn.wOut = wOut
    nn.bOut = bOut

    return nil
}

func (nn *neuralNet) backpropagate(x, y, wHidden, bHidden, wOut, bOut, output *mat.Dense) error {
    for i := 0; i < nn.config.numEpochs; i++ {
        hiddenLayerInput := new(mat.Dense)
        hiddenLayerInput.Mul(x, wHidden)
        addBHidden := func(_, col int, v float64) float64 { return v + bHidden.At(0, col) }
        hiddenLayerInput.Apply(addBHidden, hiddenLayerInput)

        hiddenLayerActivations := new(mat.Dense)
        applySigmoid := func(_, _ int, v float64) float64 { return sigmoid(v) }
        hiddenLayerActivations.Apply(applySigmoid, hiddenLayerInput)

        outputLayerInput := new(mat.Dense)
        outputLayerInput.Mul(hiddenLayerActivations, wOut)
        addBOut := func(_, col int, v float64) float64 { return v + bOut.At(0, col) }
        outputLayerInput.Apply(addBOut, outputLayerInput)
        output.Apply(applySigmoid, outputLayerInput)

        networkError := new(mat.Dense)
        networkError.Sub(y, output)

        slopeOutputLayer := new(mat.Dense)
        applySigmoidPrime := func(_, _ int, v float64) float64 { return sigmoidPrime(v) }
        slopeOutputLayer.Apply(applySigmoidPrime, output)
        slopeHiddenLayer := new(mat.Dense)
        slopeHiddenLayer.Apply(applySigmoidPrime, hiddenLayerActivations)

        dOutput := new(mat.Dense)
        dOutput.MulElem(networkError, slopeOutputLayer)
        errorAtHiddenLayer := new(mat.Dense)
        errorAtHiddenLayer.Mul(dOutput, wOut.T())

        dHiddenLayer := new(mat.Dense)
        dHiddenLayer.MulElem(errorAtHiddenLayer, slopeHiddenLayer)

        wOutAdj := new(mat.Dense)
        wOutAdj.Mul(hiddenLayerActivations.T(), dOutput)
        wOutAdj.Scale(nn.config.learningRate, wOutAdj)
        wOut.Add(wOut, wOutAdj)

        bOutAdj, err := sumAlongAxis(0, dOutput)
        if err != nil {
            return err
        }
        bOutAdj.Scale(nn.config.learningRate, bOutAdj)
        bOut.Add(bOut, bOutAdj)

        wHiddenAdj := new(mat.Dense)
        wHiddenAdj.Mul(x.T(), dHiddenLayer)
        wHiddenAdj.Scale(nn.config.learningRate, wHiddenAdj)
        wHidden.Add(wHidden, wHiddenAdj)

        bHiddenAdj, err := sumAlongAxis(0, dHiddenLayer)
        if err != nil {
            return err
        }
        bHiddenAdj.Scale(nn.config.learningRate, bHiddenAdj)
        bHidden.Add(bHidden, bHiddenAdj)
    }
    return nil
}

func (nn *neuralNet) predict(x *mat.Dense) (*mat.Dense, error) {
    if nn.wHidden == nil || nn.wOut == nil {
        return nil, errors.New("the supplied weights are empty")
    }
    if nn.bHidden == nil || nn.bOut == nil {
        return nil, errors.New("the supplied biases are empty")
    }

    output := new(mat.Dense)

    hiddenLayerInput := new(mat.Dense)
    hiddenLayerInput.Mul(x, nn.wHidden)
    addBHidden := func(_, col int, v float64) float64 { return v + nn.bHidden.At(0, col) }
    hiddenLayerInput.Apply(addBHidden, hiddenLayerInput)

    hiddenLayerActivations := new(mat.Dense)
    applySigmoid := func(_, _ int, v float64) float64 { return sigmoid(v) }
    hiddenLayerActivations.Apply(applySigmoid, hiddenLayerInput)

    outputLayerInput := new(mat.Dense)
    outputLayerInput.Mul(hiddenLayerActivations, nn.wOut)
    addBOut := func(_, col int, v float64) float64 { return v + nn.bOut.At(0, col) }
    outputLayerInput.Apply(addBOut, outputLayerInput)
    output.Apply(applySigmoid, outputLayerInput)

    return output, nil
}

func sigmoid(x float64) float64 {
  	return 1.0 / (1.0 + math.Exp(-x))
}

func sigmoidPrime(x float64) float64 {
  	return sigmoid(x) * (1.0 - sigmoid(x))
}

func sumAlongAxis(axis int, m *mat.Dense) (*mat.Dense, error) {
    numRows, numCols := m.Dims()

    var output *mat.Dense

    switch axis {
        case 0:
            data := make([]float64, numCols)
            for i := 0; i < numCols; i++ {
                col := mat.Col(nil, i, m)
                data[i] = floats.Sum(col)
            }
            output = mat.NewDense(1, numCols, data)
        case 1:
            data := make([]float64, numRows)
            for i := 0; i < numRows; i++ {
                row := mat.Row(nil, i, m)
                data[i] = floats.Sum(row)
            }
            output = mat.NewDense(numRows, 1, data)
        default:
            return nil, errors.New("invalid axis, must be 0 or 1")
    }
    return output, nil
}

Overwriting nn.go


In [227]:
!go run nn.go

10-fold Cross Validation: 
Accuracy = 0.80
Accuracy = 1.00
Accuracy = 0.27
Accuracy = 0.80
Accuracy = 0.87
Accuracy = 0.73
Accuracy = 0.73
Accuracy = 1.00
Accuracy = 0.20
Accuracy = 0.93
